In [3]:
import pandas as pd
import random
import os
import shutil
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'



In [4]:
ROOT_DIR = './'
TEST_CSV = 'test.csv'
TRAIN_CSV = 'train.csv'
ORIGIN_SUBMIT_CSV = 'sample_submission.csv'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
seed_everything(42) # Seed 고정

submit_now = datetime.datetime.now()
SUBMIT_CSV = 'sample_submission_dateNtime_%d_%d_%d_%d_%d.csv' % (submit_now.month, submit_now.day, submit_now.hour, submit_now.minute, submit_now.second)

shutil.copy(os.path.join(ROOT_DIR, ORIGIN_SUBMIT_CSV), os.path.join(ROOT_DIR, SUBMIT_CSV))



'./sample_submission_dateNtime_8_15_16_14_37.csv'

In [11]:
train_df = pd.read_csv(os.path.join(ROOT_DIR, TRAIN_CSV))
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [12]:
class LG_NRMSE(nn.Module):
    def __init__(self):
        super(LG_NRMSE, self).__init__()
        self.mse = nn.MSELoss().to(device)

    def forward(self, gt, preds):
        # 각 Y Feature별 NRMSE 총합
        # Y_01 ~ Y_08 까지 20% 가중치 부여

        all_nrmse_list = []
        # for idx in range(1,15): # ignore 'ID'
        for idx in range(1,14): # ignore ID
            rmse = torch.sqrt(self.mse(preds[:,idx], gt[:,idx]))
            nrmse = rmse / torch.mean(torch.abs(gt[:,idx]))
            all_nrmse_list.append(nrmse)
            all_nrmse = torch.stack(all_nrmse_list, dim=0)
        score = 1.2 * torch.sum(all_nrmse[:8]) + 1.0 * torch.sum(all_nrmse[8:15])
        return score

In [9]:
class Linear_net(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(Linear_net, self).__init__()
        self.dim_layer = [dim_in, 2*dim_in, 2*dim_out, dim_out]
        # self.dim_layer = [dim_in, 2*dim_in, 4*dim_in, 8*dim_in, 16*dim_in, 16*dim_out, 8*dim_out, 4*dim_out, 2*dim_out, dim_out]
        # self.dim_layer = [dim_in, 100, 200, 300, 400, 500, 400, 300, 200, 100, dim_out]
        self.layers = nn.ModuleList()
        for i in range(len(self.dim_layer) - 1):
            self.layers.append(nn.Sequential( # no ReLU, Dropout. These ruins the Result
                nn.Linear(self.dim_layer[i], self.dim_layer[i + 1]),
                nn.BatchNorm1d(self.dim_layer[i + 1])
            ))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [14]:
lr = 1 # main
# wd = 0
wd = 10e-8 # sub main
batch_size = 1024
epochs = 1000
model = Linear_net(train_x.shape[1], train_y.shape[1]).to(device)
# print(train_x.shape[1], train_y.shape[1])

optimizer = optim.Adagrad(model.parameters(), lr = lr, weight_decay = wd)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= 50, gamma=0.5)
# criterion = nn.MSELoss().to(device)
criterion = nn.L1Loss().to(device)
# criterion = LG_NRMSE().to(device)

In [19]:
# test set divisor
train_x = torch.from_numpy(train_x.to_numpy()).to(device).to(torch.float)
train_y = torch.from_numpy(train_y.to_numpy()).to(device).to(torch.float)

test_x_from_train = train_x[:train_x.shape[0] // 10]
test_y_from_train = train_y[:train_y.shape[0] // 10]
train_x = train_x[train_x.shape[0] // 10 :]
train_y = train_y[train_y.shape[0] // 10 :]

part_idx = len(train_x) // batch_size

print(train_x.shape[1], train_y.shape[1])

56 14


In [46]:
# train by k-fold
for epoch in range(epochs):
    running_loss = 0

    for iter in range(part_idx + 1):
        model.train()

        val_x_part = train_x[iter * batch_size : (iter + 1) * batch_size]
        val_y_part = train_y[iter * batch_size : (iter + 1) * batch_size]
        train_x_part = torch.cat((
            train_x[: iter * batch_size],
            train_x[(iter + 1) * batch_size :]
        ), 0)
        train_y_part = torch.cat((
            train_y[: iter * batch_size],
            train_y[(iter + 1) * batch_size :]
        ), 0)

        if iter == part_idx:
            val_x_part = train_x[iter * batch_size :]
            val_y_part = train_y[iter * batch_size :]
            train_x_part = train_x[: iter * batch_size]
            train_y_part = train_y[: iter * batch_size]
      
        pred_train = model(train_x_part)
        loss = criterion(pred_train, train_y_part)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        with torch.no_grad():
            model.eval()
            pred_val = model(val_x_part)
            val_loss = criterion(pred_val, val_y_part)
            print("Epoch : %d, Iteration : %d, Validation Loss : %f"%(epoch + 1, iter + 1, val_loss))
      
        running_loss += loss.item()

  # scheduler.step()
  
    print('--------------------------------------------')
    print('--------------------------------------------')
    print('--------------------------------------------')
    print("Epoch : %d, Loss : %f"% (epoch + 1, running_loss / (part_idx)))

  # 추가 정보
    now = datetime.datetime.now()
    EPOCH = epoch
    PATH = ROOT_DIR+"/model_epoch_%d_dateNtime_%d_%d_%d_%d_%d" % (epoch + 1, now.month, now.day, now.hour, now.minute, now.second)
    LOSS = running_loss

    torch.save({
              'epoch': EPOCH,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': LOSS,
              }, PATH)

Epoch : 1, Iteration : 1, Validation Loss : 9.732940
Epoch : 1, Iteration : 2, Validation Loss : 9.545956
Epoch : 1, Iteration : 3, Validation Loss : 9.457129
Epoch : 1, Iteration : 4, Validation Loss : 9.244581
Epoch : 1, Iteration : 5, Validation Loss : 9.171203
Epoch : 1, Iteration : 6, Validation Loss : 9.087253
Epoch : 1, Iteration : 7, Validation Loss : 8.998631
Epoch : 1, Iteration : 8, Validation Loss : 8.924212
Epoch : 1, Iteration : 9, Validation Loss : 8.740838
Epoch : 1, Iteration : 10, Validation Loss : 8.756976
Epoch : 1, Iteration : 11, Validation Loss : 8.517912
Epoch : 1, Iteration : 12, Validation Loss : 8.430731
Epoch : 1, Iteration : 13, Validation Loss : 8.487666
Epoch : 1, Iteration : 14, Validation Loss : 8.394745
Epoch : 1, Iteration : 15, Validation Loss : 8.165771
Epoch : 1, Iteration : 16, Validation Loss : 8.087575
Epoch : 1, Iteration : 17, Validation Loss : 8.128859
Epoch : 1, Iteration : 18, Validation Loss : 8.074234
Epoch : 1, Iteration : 19, Validation

KeyboardInterrupt: 

In [5]:
test_x = pd.read_csv(os.path.join(ROOT_DIR, TEST_CSV)).drop(columns=['ID'])
test_x = torch.from_numpy(test_x.to_numpy()).to(device).to(torch.float)

with torch.no_grad():
  model.eval()
  preds = model(test_x)
  preds = preds.cpu().detach().numpy()

print('Done.')

NameError: name 'model' is not defined

In [48]:
submit = pd.read_csv(os.path.join(ROOT_DIR, SUBMIT_CSV))
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')
submit.to_csv(os.path.join(ROOT_DIR, SUBMIT_CSV), index=False)

Done.


In [6]:
os.path.join(ROOT_DIR, SUBMIT_CSV)

'./sample_submission_dateNtime_8_15_16_14_37.csv'

In [7]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    # for idx in range(1,15): # ignore 'ID'
    for idx in range(1,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [30]:
test_model = Linear_net(train_x.shape[1], train_y.shape[1])
optimizer = optim.Adagrad(test_model.parameters(), lr = lr, weight_decay = wd)

############################################################ fix this path
PATH = 'model_epoch_676_dateNtime_8_15_16_4_53'
# PATH = "/content/gdrive/MyDrive/lgaimer_model/model_epoch_%d_dateNtime_%d_%d_%d_%d_%d" % (epochs, now.month, now.day, now.hour, now.minute, now.second)

checkpoint = torch.load(PATH)
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

test_model.eval()
print(" ")
# test_model.train()

In [32]:
test_model.cuda()

Linear_net(
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=56, out_features=112, bias=True)
      (1): BatchNorm1d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=112, out_features=28, bias=True)
      (1): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=28, out_features=14, bias=True)
      (1): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [36]:
test = test_model(test_x_from_train)

In [34]:
score = lg_nrmse(test_y_from_train.cpu().detach().numpy(), test_model(test_x_from_train).cpu().detach().numpy())


In [35]:
score

1.8308801531791687

In [39]:
test

tensor([[  1.3417,   1.0335,   0.9849,  ..., -26.2215, -26.2172, -26.2277],
        [  1.3421,   1.0339,   0.9858,  ..., -26.2206, -26.2164, -26.2269],
        [  1.3434,   1.0354,   0.9855,  ..., -26.2189, -26.2147, -26.2253],
        ...,
        [  1.3552,   1.0495,   0.9921,  ..., -26.2026, -26.1981, -26.2099],
        [  1.3545,   1.0486,   0.9916,  ..., -26.2036, -26.1991, -26.2108],
        [  1.3539,   1.0478,   0.9914,  ..., -26.2044, -26.1999, -26.2116]],
       device='cuda:0', grad_fn=<NativeBatchNormBackward>)

In [43]:
shutil.copy(os.path.join(ROOT_DIR, ORIGIN_SUBMIT_CSV), os.path.join(ROOT_DIR, SUBMIT_CSV))



'./sample_submission_dateNtime_8_15_16_14_37.csv'

In [ ]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'./sample_submission_dateNtime_8_15_16_3_32.csv', 
'', 
'235927', 
'indexx', 
'etr' )